In [2]:
"""
用于统计某个时间段内股票连板（仅指涨停）的次数、断板时间、断板最高价
大致分成如下部分：
1.数据导入，导入的数据df中应该至少包含：收盘价、股票代码、日期、最高限价。另还需要准备一个好的交易所日历（聚宽获取）
2.生成up_limited，并按照股票代码分组生成stock_groups，用于统计每个股票在某段时间内出现涨停的日期
3.统计股票连板（仅指涨停）的次数、断板时间、断板最高价，生成一个列表，每个元素都是字典，key是股票代码，value是连板次数及断板最高价
4.列表转化为df，并合并最高限价数据
"""
import pandas as pd
import numpy as np
import requests
import re
from tqdm import tqdm
import selenium
import pymysql
from datetime import datetime
import schedule
import time
import tushare as ts
import os
import glob

In [12]:
#设置文件夹路径
folder_path = r"./历史股价/"
# 使用glob查找所有xlsx文件
file_paths = glob.glob(f"{folder_path}/*.xlsx")

# 创建一个空的列表，用于存放每个文件读取后的DataFrame
dataframes = []

# 使用tqdm显示进度条
for file in tqdm(file_paths, desc="Processing files"):
    # 读取每个Excel文件
    df = pd.read_excel(file)
    # 将每个DataFrame添加到列表中
    dataframes.append(df)

# 将所有DataFrame按行合并
combined_df = pd.concat(dataframes, ignore_index=True)

# 按照'time'和'code'列进行排序
df= combined_df.sort_values(by=['time', 'code'])
del combined_df

df

Processing files: 100%|████████████████████████████| 80/80 [03:00<00:00,  2.25s/it]


,time,code,open,close,volume,money,paused,high_limit,low_limit
172674,2020-01-02,000001.XSHE,14.54,14.73,175282129,2.571196e+09,0,15.80,12.93
172688,2020-01-02,000002.XSHE,26.79,26.59,123917223,3.342374e+09,0,28.91,23.65
172702,2020-01-02,000006.XSHE,4.84,4.81,13994241,6.745287e+07,0,5.26,4.30
172716,2020-01-02,000008.XSHE,3.65,3.70,37874313,1.392082e+08,0,3.98,3.26
172730,2020-01-02,000009.XSHE,6.20,6.57,243821393,1.559130e+09,0,6.72,5.50
...,...,...,...,...,...,...,...,...,...
599435,2024-07-29,688368.XSHG,57.08,55.56,657206,3.719380e+07,0,68.94,45.96
599444,2024-07-29,688369.XSHG,14.83,14.60,1877546,2.736828e+07,0,17.80,11.86
599453,2024-07-29,688388.XSHG,9.28,9.07,4199968,3.820690e+07,0,11.06,7.38
599462,2024-07-29,688389.XSHG,15.11,14.73,2112541,3.118153e+07,0,18.07,12.05


In [23]:
from datetime import datetime

up_limited = df[
    (df['close'] == df['high_limit']) &
    (df['paused'] == 0) &
    (df['time'] >= datetime.strptime('2023-01-01', '%Y-%m-%d')) 
      # 修正格式字符串
]

date_tuple = pd.read_excel(r"./日历（深圳）.xlsx")
# 取并集并排序
date_tuple  = sorted(set(df['time']).union(set(date_tuple['day'])))

# 将结果转换为元组
date_tuple = tuple(date_tuple )
up_limited

,time,code,open,close,volume,money,paused,high_limit,low_limit
1603205,2023-01-03,000032.XSHE,20.35,21.96,40352735,8.503231e+08,0,21.96,17.97
1603303,2023-01-03,000045.XSHE,12.05,12.05,1482139,1.786000e+07,0,12.05,9.86
1603905,2023-01-03,000419.XSHE,8.11,8.11,143967012,1.166840e+09,0,8.11,6.63
1604969,2023-01-03,000607.XSHE,5.30,5.88,133842865,7.559915e+08,0,5.88,4.81
1605333,2023-01-03,000665.XSHE,6.54,7.19,134617751,9.248166e+08,0,7.19,5.89
...,...,...,...,...,...,...,...,...,...
596870,2024-07-29,603569.XSHG,6.60,7.25,13849777,9.611025e+07,0,7.25,5.93
597032,2024-07-29,603602.XSHG,11.96,13.24,16289301,2.110484e+08,0,13.24,10.84
597635,2024-07-29,603716.XSHG,5.79,6.49,26739158,1.651429e+08,0,6.49,5.31
598076,2024-07-29,603826.XSHG,25.76,28.34,3627844,1.008436e+08,0,28.34,23.18


In [67]:

stock_groups=up_limited.groupby('code')
continue_dict = {}
continue_list = []

for stock,group in tqdm(stock_groups):

    if len(group)<=1:
        continue
    tem_num=1
    real_num=1
    l=0
    day_break=[]
    # print(stock)
    
    time_list = list(group['time'])
    all_time = list(date_tuple)
    
    
    all_idx = 0
    continue_count = 1
    # end_time = None
    coutinue_dict = {}
    
    for time_idx in range(len(time_list)-1):

        time = time_list[time_idx]
        
        while(all_time[all_idx] != time):
            all_idx += 1
            
        aft_time = time_list[time_idx + 1]
        
        if all_time[all_idx + 1] == aft_time:
            continue_count += 1

            
            
        elif continue_count > 1:
            continue_dict = {'stock': stock, 'day_nums': continue_count, 'end_time': time}
            continue_list.append(continue_dict)
            continue_count = 1
    if continue_count>1:
        continue_dict={'stock': stock, 'day_nums': continue_count, 'end_time': time_list[-1]}
        continue_list.append(continue_dict)
        
            

    



100%|████████████████████████████████████████| 2312/2312 [00:00<00:00, 9654.82it/s]

7
Aft_time: 2023-01-05 00:00:00
Bll_time: 2023-01-05 00:00:00
count_time: 2
7
{'stock': '000938.XSHE', 'day_nums': 2, 'end_time': Timestamp('2023-01-05 00:00:00')}
7
7
7
7
Aft_time: 2024-03-01 00:00:00
Bll_time: 2024-03-01 00:00:00
count_time: 2
7
{'stock': '000938.XSHE', 'day_nums': 2, 'end_time': Timestamp('2024-03-01 00:00:00')}


In [68]:
df = pd.DataFrame(continue_list)
df